# Direction

-   Write a Python code to answer each question.

-   Make at least some simple comment (`# ...`) in each question.

-   Use your working directory with the subfolder, `data`, so that the relative pathname of CSV files in the subfolder `data` is sufficient to import the CSV files.

# Load Libraries

-   Import all the Python libraries we need here.


In [ ]:
import pandas as pd

# Question 1

-   Consider the probability of survival in `titanic_1.csv`.


In [ ]:
#| echo: true
#| eval: true
titanic_1 = pd.read_csv('https://bcdanl.github.io/data/titanic_1.csv')

In [ ]:
#| echo: true
#| eval: true
titanic_1.head()

In [ ]:
#| echo: true
#| eval: true
titanic_1.describe()

## Q1a

-   Calculate the simple difference between the probability of survival when passengers are first-class and the probability of survival when they are not.


In [ ]:
# Count the number of passengers in each class and return the count in descending order
titanic_1[['pclass']].value_counts()

# Count the number of passengers who survived and who didn't and return the count in descending order
titanic_1[['survived']].value_counts()

# Add a new column 'd' to the titanic_1 dataframe and set all values to 0
titanic_1['d'] = 0

# For rows where the 'pclass' column is '1st class', set the value of the 'd' column to 1
titanic_1.loc[titanic_1['pclass']=='1st class', 'd'] = 1

# Add a new column 'survived_d' to the titanic_1 DataFrame and set all values to 0
titanic_1['survived_d'] = 0

# For rows where the 'survived' column is 'yes', set the value of the 'survived_d' column to 1
titanic_1.loc[titanic_1['survived']=='yes', 'survived_d'] = 1

# Compute the mean of 'survived_d' for rows where 'd' is 0
e_y0 = titanic_1.loc[titanic_1['d']==0, 'survived_d'].mean()

# Compute the mean of 'survived_d' for rows where 'd' is 1
e_y1 = titanic_1.loc[titanic_1['d']==1, 'survived_d'].mean()

# Compute the SDP (so called treatment effect) by subtracting the mean of 'survived_d' where 'd' is 0 from the mean where 'd' is 1
SDP = e_y1 - e_y0

# Return the value of SDP
SDP

## Q1b

-   How much does the probability of survival increase for first-class passengers relative to those who are not first-class passengers?

-   `SDP` tells us what would happen to the probability of survival if non-first-class passengers were first-class.

    -   In other words, `SDP` means the effect of being the first-class on the probability of survival from the Titanic Disaster.

## Q1c

-   Consider the probability of survival in `titanic_2.csv`.


In [ ]:
titanic_2 = pd.read_csv('https://bcdanl.github.io/data/titanic_2.csv')

In [ ]:
titanic_2.head()

In [ ]:
titanic_2.describe()

-   After stratifying on gender and age, what happens to the difference in the probabilities of survival between first-class passengers and non-first-class passengers.

-   Explain in your own words what stratifying on gender and age did for this difference in probabilities of survival between first-class passengers and non-first-class passengers.


In [ ]:
# Get count of passengers by pclass
titanic_2[['pclass']].value_counts()

# Get count of passengers who survived or not
titanic_2[['survived']].value_counts()

# Get count of passengers by gender
titanic_2[['sex']].value_counts()

# Get count of passengers by age
titanic_2[['age']].value_counts()

In [ ]:
titanic_2.groupby('sex').describe()

In [ ]:
# Create a new column 'd' and set its value to 0 for all rows
titanic_2['d'] = 0

# Set the value of column 'd' to 1 for rows where pclass is '1st class'
titanic_2.loc[titanic_2['pclass']=='1st class', 'd'] = 1

# Create a new column 'survived_d' and set its value to 0 for all rows
titanic_2['survived_d'] = 0

# Set the value of column 'survived_d' to 1 for rows where survived is 'yes'
titanic_2.loc[titanic_2['survived']=='yes', 'survived_d'] = 1

# Create a new column 'sex_d' and set its value to 0 for all rows
titanic_2['sex_d'] = 0

# Set the value of column 'sex_d' to 1 for rows where sex is 'male'
titanic_2.loc[titanic_2['sex']=='male', 'sex_d'] = 1

# Create a new column 'AgeGroup' and set its value to 0 for all rows
titanic_2['AgeGroup'] = 0

# Set the value of column 'AgeGroup' to 1 for rows where age is greater than or equal to 18
titanic_2.loc[titanic_2['age'] >= 18, 'AgeGroup'] = 1

In [ ]:
titanic_2.groupby('AgeGroup').describe()

In [ ]:
titanic_2.groupby(['sex', 'AgeGroup']).describe()

In [ ]:
# Create a new column 's' and set its value to 0 for all rows
titanic_2['s'] = 0 

# Set the value of column 's' based on gender and age group
titanic_2.loc[(titanic_2.sex_d == 0) & (titanic_2.AgeGroup == 1), 's'] = 1
titanic_2.loc[(titanic_2.sex_d == 0) & (titanic_2.AgeGroup == 0), 's'] = 2
titanic_2.loc[(titanic_2.sex_d == 1) & (titanic_2.AgeGroup == 1), 's'] = 3
titanic_2.loc[(titanic_2.sex_d == 1) & (titanic_2.AgeGroup == 0), 's'] = 4

# Get the number of observations where d is 0
obs = titanic_2.loc[titanic_2.d == 0].shape[0]

# Define a function to calculate weighted average effect
def weighted_avg_effect(df):
  
  # Calculate the difference in survival rates between the treatment and control groups
    diff = ( df[ df.d == 1 ].survived_d.mean() - 
             df[ df.d == 0 ].survived_d.mean() )
  
  # Calculate the weight assigned to the treatment group
    weight = df[ df.d == 0 ].shape[0] / obs
    
  # Calculate the weighted average effect
    return diff * weight


# Apply the weighted_avg_effect function to each group in the data frame grouped by the s variable
SDP2 = titanic_2.groupby('s').apply( weighted_avg_effect )

# Calculate the weighted average effect of treatment for the entire population
WSDP = SDP2.sum()

WSDP

-   The probability of survival for the first-class passengers can be different across gender and age groups.
    -   In other words, the effect of being the first-class on the probability of survival from the Titanic Disaster can be different across genders and age groups.
-   WSDP takes into account the difference in the effect of being first-class across gender and age groups by weighting.


In [ ]:
WSDP - SDP

-   The probability of survival for first-class after taking into account gender and age (`WSDP`) is less than that (`SDP`) which does presumably assume that characteristics of passengers such as gender and ages is related with the probability of survival.

# Question 2

-   Consider the data file, `NHL-1617.csv`.


In [ ]:
#| echo: true
#| eval: true
nhl1617 = pd.read_csv('https://bcdanl.github.io/data/NHL-1617.csv')

In [ ]:
#| echo: true
#| eval: true
nhl1617.head()

In [ ]:
#| echo: true
#| eval: true
nhl1617.describe()

## Q2a

-   How many players have been recorded?


In [ ]:
# the number of unique players
q2a = nhl1617['id_player'].nunique()  

q2a

## Q2b.

-   A column `points` ("`P`") is missing in the data. The number of points of a player is defined as the sum of his goals ("`G`") and `assists` ("`A`").

-   Add the point column "`P`" to your DataFrame.


In [ ]:
# create a new column called 'P' in the nhl1617 dataframe that is the sum of the 'G' and 'A' columns
q2b = nhl1617.assign(P = nhl1617['G'] + nhl1617['A'])

q2b

## Q2c.

-   Who is the top scorer in terms of points?


In [ ]:
# Sort the DataFrame q2b by the column 'P' in descending order
q2c = q2b.sort_values(by= 'P', ascending=False)

q2c

## Q2d.

-   How many Russian (non-goalie) players had some ice time in there 2016/2017 regular season?

-   Hint: Nationality of a player can be found in "Nat". Russians are indicated by "RUS".


In [ ]:
# the number of rows in the 'nhl1617' dataframe where the value in the 'Nat' column is equal to 'RUS', and returns the count as output.
q2d = len( nhl1617[nhl1617['Nat'] == 'RUS'] )

q2d

## Q2e.

-   What are their names?


In [ ]:
# Select rows where the 'Nat' column equals 'RUS', then select only the 'Last_Name' and 'First_Name' columns
q2e = nhl1617[ nhl1617['Nat'] == 'RUS' ][ ['Last_Name', 'First_Name'] ]

q2e

## Q2f.

-   Who performed best among the Russian players in terms of points ("`P`")?


In [ ]:
# Select only the rows where 'Nat' column is 'RUS'
# Then sort these rows in descending order based on the values in the 'P' column
q2f = q2b[q2b['Nat'] == 'RUS'].sort_values(by='P', ascending = False)

q2f

## Q2g.

-   How many points ("`P`") did he have?


In [ ]:
q2g = (
  q2b[q2b['Nat'] == 'RUS'] 
  .sort_values(by='P', ascending=False)
  .head(1)[['Last_Name', 'First_Name', 'P']]
  )
  
q2g

-   The above code creates a new DataFrame called `q2g`, which:

    1.  Filters `q2b` for players whose nationality is '`RUS`'
    2.  Sorts the filtered DataFrame by the '`P`' column in descending order
    3.  Selects the top row of the sorted DataFrame using the `.head(1)` method
    4.  Filters the columns '`Last_Name`', '`First_Name`', and '`P`' from the selected row

## Q2h.

-   How well did he perform in the entire league? Put differently, what was his rank in terms of points?


In [ ]:
q2h = (
     q2b.assign(ranking = q2b['P'].rank(method = 'min', ascending=False))  # Create a new column 'ranking' based on the values in column 'P'
        .sort_values(by='P', ascending=False)  # Sort the DataFrame by 'P' column in descending order
        .loc[q2b['Nat'] == 'RUS']  # Filter the rows where the 'Nat' column is 'RUS'
        .head(1)[['ranking', 'Last_Name', 'First_Name', 'P']]  # Select the top row and specific columns
        )

q2h

## Q2i.

-   Find the top ten scorers (in terms of points) and print them including their number of point and their respective team.


In [ ]:
q2i = ( q2b.assign( ranking = q2b['P'].rank(ascending=False) )   # add a new column 'ranking' based on the 'P' column's rank
           .sort_values(by='P', ascending=False)                  # sort by 'P' column in descending order
           )

( q2i.loc[q2i['ranking'] <= 10]                                    # select rows where 'ranking' is less than or equal to 10
            [['ranking', 'Last_Name', 'First_Name', 'P', 'Team']]  # select specific columns
            )


## Q2j.

-   What are the three countries with the most players originating from?


In [ ]:
# Create a dataframe of count of players by nationality
q2j = q2b[['Nat']].value_counts().reset_index()

# Rename the columns of the dataframe
q2j.columns = ['Nat', 'counts']

# Create a new column 'ranking' with the ranking of each nationality by count
q2j = ( q2j.assign( ranking = q2j['counts']
                             .rank(method = 'dense', ascending=False) )
           
           # Filter the dataframe to only include the top 3 nationalities by count
           .query('ranking <= 3' ) 
      )

# Question 3

-   Consider the data file, `coal_production_county.csv`.


In [ ]:
#| echo: true
#| eval: true
coal = pd.read_csv('https://bcdanl.github.io/data/coal_production_county.csv')

In [ ]:
#| echo: true
#| eval: true
coal.describe()

In [ ]:
#| echo: true
#| eval: true
coal.groupby('year').describe()

In [ ]:
coal.groupby('state').describe()

In [ ]:
coal.groupby(['year', 'state']).describe()

## Q3a.

-   For each type of mine, calculate the total coal production for each pair of state-year.


In [ ]:
# Create a new column 'production' which is the sum of two other columns
coal['production'] = coal['production_underground'] + coal['production_surface']

# Group the data by 'state' and 'year' columns and calculate the sum of 'production', 'production_underground', and 'production_surface'
q3a = (
       coal[ ['state', 'year', 'production',
             'production_underground',
             'production_surface'] ]
       .groupby(['state', 'year'])
       .sum()
       )

q3a

## Q3b.

-   Find the top 5 coal-producing states for each year.


In [ ]:
# Creates a new dataframe q3b
q3b = (
       # Reset the index of q3a
       q3a.reset_index()
       # Sort the values of q3a by the production column in descending order
       .sort_values(['production'], ascending = False)
       # Group q3a by year and get the head with the largest production for each year
       .groupby('year')
       .head()
       # Sort q3b first by year in ascending order and then by production in descending order
       .sort_values(['year','production'], ascending = [True, False])
       )

q3b

## Q3c.

-   Visualize the yearly trend of the total coal production from each type of mine.


In [ ]:
# selecting the columns 'year', 'production_underground', and 'production_surface' from the original DataFrame coal, grouping them by year, and summing the values of each group.
q3c = (
  coal[['year','production_underground', 'production_surface']]
    .groupby(['year']).sum()
    )

In [ ]:
# create a line plot for the 'production_underground' column of the q3c DataFrame.
q3c['production_underground'].plot()

In [ ]:
# create a line plot for the 'production_surface' column of the q3c DataFrame.
q3c['production_surface'].plot()

In [ ]:
# create a line plot for the 'production_underground' and 'production_surface' columns of the q3c DataFrame on the same graph.
q3c.plot()